In [8]:
# Create Spark Session for Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
#Read CSV File via spark 
dt = spark.read.csv('ws2_data.csv', header = True, inferSchema = True, )

In [12]:
# show data information 5 row
dt.show(5)

+-------------------+--------+-------+------------+-----+
|          timestamp| user_id|book_id|     country|price|
+-------------------+--------+-------+------------+-----+
|2021-05-01 08:03:44|824dc295|      2|      Israel|24.95|
|2021-05-02 02:48:34|f80fe106|      6|    Thailand|31.49|
|2021-05-01 16:54:57|01cb9fe0|      9|       Italy|29.65|
|2021-05-01 21:45:28|59508f47|     13|South Africa| 22.4|
|2021-05-01 14:50:52|3553ebbb|     14|      Canada| 28.0|
+-------------------+--------+-------+------------+-----+
only showing top 5 rows



In [15]:
#Check type of colum
dt.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- price: double (nullable = true)



In [17]:
# count row and column
print((dt.count(), len(dt.columns)))

(500000, 5)


In [20]:
#Statistics
dt.summary().show()

+-------+-----------+-----------------+-------+------------------+
|summary|    user_id|          book_id|country|             price|
+-------+-----------+-----------------+-------+------------------+
|  count|     499998|           500000| 500000|            500000|
|   mean|   Infinity|       740.377786|   null|31.459311679998564|
| stddev|        NaN|559.9366172146358|   null|10.945931459891698|
|    min|   00000f49|                1|Algeria|               0.0|
|    25%|2.4050123E7|              262|   null|             24.95|
|    50%|  7.04742E7|              614|   null|             29.99|
|    75%|   4.857E81|             1215|   null|             34.99|
|    max|   ffffe3e7|             2269|  Yemen|              84.0|
+-------+-----------+-----------------+-------+------------------+



In [21]:
#find data missing 
dt.summary("count").show()

+-------+-------+-------+-------+------+
|summary|user_id|book_id|country| price|
+-------+-------+-------+-------+------+
|  count| 499998| 500000| 500000|500000|
+-------+-------+-------+-------+------+



In [22]:
#user_id colum have missing data 
dt.where(dt.user_id.isNull()).show()

+-------------------+-------+-------+-------+-----+
|          timestamp|user_id|book_id|country|price|
+-------------------+-------+-------+-------+-----+
|2021-05-01 14:41:20|   null|    516| Brazil|27.37|
|2021-06-30 02:27:00|   null|   1390|  Japan|27.99|
+-------------------+-------+-------+-------+-----+

